## Imports

In [27]:
import sys, os, platform
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## Loading Data

In [33]:
breast_cancer = pd.read_csv("data/breastcancer_processed.csv", lineterminator='\r')
display(breast_cancer.head())

,Benign,ClumpThickness,UniformityOfCellSize,UniformityOfCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses
0,0,5,1,1,1,2,1,3,1,1
1,0,5,4,4,5,7,10,3,2,1
2,0,3,1,1,1,2,2,3,1,1
3,0,6,8,8,1,3,4,3,7,1
4,0,4,1,1,3,2,1,3,1,1


## Train Logistic Regression to get Weights

We need the weights $w$ to calculate the score $Score(x) = w \cdot x$.
The target is 'Benign'. In this dataset: 0 = Malignant, 1 = Benign.

Let's assume the goal is to explain why someone is more likely to be Benign (or Malignant).

In [29]:
print("Classes:", breast_cancer["Benign"].unique())
# 0 usually Malignant, 1 Benign in this specific dataset context often.

# Prepare Data
X = breast_cancer.drop(columns=["Benign"])
y = breast_cancer["Benign"]

# Train Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(fit_intercept=True, max_iter=1000)
clf.fit(X_train, y_train)

weights = dict(zip(X.columns, clf.coef_[0]))
intercept = clf.intercept_[0]

print("Weights:")
for f, w in weights.items():
    print(f"{f}: {w:.4f}")

print(f"\nIntercept: {intercept:.4f}")

Classes: [0 1]
Weights:
ClumpThickness: 0.4520
UniformityOfCellSize: -0.0501
UniformityOfCellShape: 0.3631
MarginalAdhesion: 0.2887
SingleEpithelialCellSize: 0.0625
BareNuclei: 0.3617
BlandChromatin: 0.5104
NormalNucleoli: 0.1943
Mitoses: 0.3819

Intercept: -9.6454


## Rank Candidates based on Model Probability

Score = $w \cdot x + b$. Higher score = higher probability of class 1 (Benign).

In [30]:
breast_cancer["Score"] = clf.decision_function(X)
ranked_df = breast_cancer.sort_values("Score", ascending=False).reset_index(drop=True)

print("Top 5 (Most likely Benign):")
display(ranked_df.head())

print("Bottom 5 (Most likely Malignant):")
display(ranked_df.tail())

Top 5 (Most likely Benign):


,Benign,ClumpThickness,UniformityOfCellSize,UniformityOfCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Score
0,1,8,10,10,10,6,10,10,10,10,14.846007
1,1,10,10,10,10,5,10,10,10,7,14.542011
2,1,9,10,10,10,10,5,10,10,10,13.739131
3,1,10,10,10,7,9,10,7,10,10,13.540229
4,1,6,10,10,10,10,10,8,10,10,13.171054


Bottom 5 (Most likely Malignant):


,Benign,ClumpThickness,UniformityOfCellSize,UniformityOfCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Score
678,0,1,1,1,1,2,1,1,1,1,-7.018427
679,0,1,1,1,1,1,1,1,1,1,-7.080884
680,0,1,1,1,1,1,1,1,1,1,-7.080884
681,0,1,1,1,1,1,1,1,1,1,-7.080884
682,0,1,1,1,1,1,1,1,1,1,-7.080884


## Prepare for Explanation

Select Rank #1 (Winner) and Rank #2 (Runner-up) to explain why Winner > Runner-up.

In [31]:
X_row = ranked_df.iloc[0]
Y_row = ranked_df.iloc[1]

# Features map
features = [c for c in breast_cancer.columns if c not in ["Benign", "Score"]]

print(f"Comparing X (Rank 1) vs Y (Rank 2)")
print(f"Score X: {X_row['Score']:.4f}")
print(f"Score Y: {Y_row['Score']:.4f}")

df_scores = pd.DataFrame({
    "feature": features,
    "weight": [weights[f] for f in features],
    "x": [X_row[f] for f in features],
    "y": [Y_row[f] for f in features]
})

df_scores["delta"] = df_scores["weight"] * (df_scores["x"] - df_scores["y"])
df_scores["sign"] = np.where(df_scores["delta"] > 0, "pro", np.where(df_scores["delta"] < 0, "con", "neutral"))

display(df_scores)

Comparing X (Rank 1) vs Y (Rank 2)
Score X: 14.8460
Score Y: 14.5420


,feature,weight,x,y,delta,sign
0,ClumpThickness,0.452039,8.0,10.0,-0.904078,con
1,UniformityOfCellSize,-0.050094,10.0,10.0,-0.000000,neutral
2,UniformityOfCellShape,0.363113,10.0,10.0,0.000000,neutral
3,MarginalAdhesion,0.288723,10.0,10.0,0.000000,neutral
4,SingleEpithelialCellSize,0.062458,6.0,5.0,0.062458,pro
5,BareNuclei,0.361749,10.0,10.0,0.000000,neutral
6,BlandChromatin,0.510353,10.0,10.0,0.000000,neutral
7,NormalNucleoli,0.194321,10.0,10.0,0.000000,neutral
8,Mitoses,0.381872,10.0,7.0,1.145616,pro


## Advanced Solvers (1-m), (m-1), (m-n)

The project requires:
1. **(1-m)**: One Pro covers multiple Cons.
2. **(m-1)**: Multiple Pros cover one Con.
3. **(m-n)**: General case (many-to-many).

In [ ]:
def solve_explanation_general(delta, pros, cons, type="m-n", verbose=False):
    # Types: "1-1", "1-m", "m-1", "m-n"
    
    m = gp.Model(f"explanation_{type}")
    m.Params.OutputFlag = 1 if verbose else 0
    
    P = len(pros)
    C = len(cons)

    K = C # Max possible groups
    
    # x[c, k] = 1 if con c is in group k
    x = m.addVars(C, K, vtype=GRB.BINARY, name="x")
    # y[p, k] = 1 if pro p is in group k
    y = m.addVars(P, K, vtype=GRB.BINARY, name="y")
    # u[k] = 1 if group k is used
    u = m.addVars(K, vtype=GRB.BINARY, name="u")

    # 1. Every Con must be in exactly one used group
    for c_idx in range(C):
        m.addConstr(gp.quicksum(x[c_idx, k] for k in range(K)) == 1)

    # 2. Every Pro in at most one used group
    for p_idx in range(P):
        m.addConstr(gp.quicksum(y[p_idx, k] for k in range(K)) <= 1)

    # 3. If a group is used, it must be valid (margin > 0)
    vals_p = [delta[p] for p in pros]
    vals_c = [delta[c] for c in cons]
    BigM = 10000
    epsilon = 0.0001

    for k in range(K):
        margin_k = gp.quicksum(vals_p[i] * y[i, k] for i in range(P)) + \
                   gp.quicksum(vals_c[j] * x[j, k] for j in range(C))
        
        # Linking u[k]: u[k] >= x[c, k]
        for c_idx in range(C):
            m.addConstr(u[k] >= x[c_idx, k])
        
        m.addConstr(margin_k >= epsilon - BigM * (1 - u[k]))

    # TYPE CONSTRAINTS
    if type == "1-1":
        for k in range(K):
            m.addConstr(gp.quicksum(y[i, k] for i in range(P)) <= 1)
            m.addConstr(gp.quicksum(x[j, k] for j in range(C)) <= 1)
    elif type == "1-m":
        for k in range(K):
            m.addConstr(gp.quicksum(y[i, k] for i in range(P)) <= 1)
    elif type == "m-1":
        for k in range(K):
            m.addConstr(gp.quicksum(x[j, k] for j in range(C)) <= 1)
    elif type == "m-n":
        pass

    total_margin = gp.quicksum(
        (gp.quicksum(vals_p[i] * y[i, k] for i in range(P)) + 
         gp.quicksum(vals_c[j] * x[j, k] for j in range(C))) 
        for k in range(K)
    )
    m.setObjective(total_margin, GRB.MAXIMIZE)

    m.optimize()

    if m.Status == GRB.OPTIMAL:
        explanation = []
        for k in range(K):
            if u[k].X > 0.5:
                pk = [pros[i] for i in range(P) if y[i, k].X > 0.5]
                ck = [cons[j] for j in range(C) if x[j, k].X > 0.5]
                if ck: 
                    marg = sum(delta[p] for p in pk) + sum(delta[c] for c in ck)
                    explanation.append({"Pros": pk, "Cons": ck, "Margin": marg})
        return {"status": "feasible", "explanation": explanation}
    else:
        return {"status": "infeasible", "message": "Certificate of Non-Existence: No valid trade-off structure found."}

pros = df_scores.loc[df_scores["delta"] > 0, "feature"].tolist()
cons = df_scores.loc[df_scores["delta"] < 0, "feature"].tolist()
delta_map = dict(zip(df_scores["feature"], df_scores["delta"]))

print("---- (1-1) Explanation ----")
res11 = solve_explanation_general(delta_map, pros, cons, type="1-1")
print(res11["status"])
if res11["status"] == "feasible":
    display(pd.DataFrame(res11["explanation"]))

print("\n---- (1-m) Explanation ----")
res1m = solve_explanation_general(delta_map, pros, cons, type="1-m")
print(res1m["status"])
if res1m["status"] == "feasible":
    display(pd.DataFrame(res1m["explanation"]))

print("\n---- (m-1) Explanation ----")
resm1 = solve_explanation_general(delta_map, pros, cons, type="m-1")
print(resm1["status"])
if resm1["status"] == "feasible":
    display(pd.DataFrame(resm1["explanation"]))

print("\n---- (m-n) Explanation ----")
resmn = solve_explanation_general(delta_map, pros, cons, type="m-n")
print(resmn["status"])
if resmn["status"] == "feasible":
    display(pd.DataFrame(resmn["explanation"]))

---- (1-1) Explanation ----
feasible


,Pros,Cons,Margin
0,[Mitoses],[ClumpThickness],0.241538



---- (1-m) Explanation ----
feasible


,Pros,Cons,Margin
0,[Mitoses],[ClumpThickness],0.241538



---- (m-1) Explanation ----
feasible


,Pros,Cons,Margin
0,"[SingleEpithelialCellSize, Mitoses]",[ClumpThickness],0.303996



---- (m-n) Explanation ----
feasible


,Pros,Cons,Margin
0,"[SingleEpithelialCellSize, Mitoses]",[ClumpThickness],0.303996


## Additional Verification Examples
Here we demonstrate the robustness of the explanation system by comparing other pairs of candidates, such as mid-tier rankings and top-tier differences.

In [ ]:
def explain_pair(rank_a, rank_b, solvers_list=["1-1", "1-m", "m-1", "m-n"]):
    row_a = ranked_df.iloc[rank_a]
    row_b = ranked_df.iloc[rank_b]
    
    print(f"\n{'='*60}")
    print(f"Comparing Rank #{rank_a+1} (Score {row_a['Score']:.2f}) vs Rank #{rank_b+1} (Score {row_b['Score']:.2f})")
    print(f"{'='*60}")
    
    # Recalculate Deltas for this specific pair
    features = [c for c in breast_cancer.columns if c not in ["Benign", "Score"]]
    delta_map = {}
    pros = []
    cons = []
    
    for f in features:
        val_a = row_a[f]
        val_b = row_b[f]
        w = weights[f]
        d = w * (val_a - val_b)
        delta_map[f] = d
        if d > 1e-6:
            pros.append(f)
        elif d < -1e-6:
            cons.append(f)
            
    print(f"Pros: {len(pros)}, Cons: {len(cons)}")
    
    # specific check for 1-1 to be rigorous
    # if 1-1 is feasible, we stop? or we show all? 
    # The prompt asks for verification examples, so let's try strict order or just show what works.
    
    for solver_type in solvers_list:
        print(f"\n--- Attempting ({solver_type}) ---")
        res = solve_explanation_general(delta_map, pros, cons, type=solver_type)
        if res["status"] == "feasible":
            print("SUCCESS")
            display(pd.DataFrame(res["explanation"]))
            # Optional: break if you only want the simplest explanation
            # break 
        else:
            print(f"FAILED: {res['message']}")


In [ ]:
# Example 1: Mid-tier comparison (Rank 10 vs Rank 11)
# These candidates often have very similar scores, making explanation difficult.
explain_pair(9, 10, solvers_list=["1-1", "1-m", "m-1", "m-n"])


In [ ]:
# Example 2: Top-tier comparison (Rank 1 vs Rank 5)
# A slightly larger gap than 1 vs 2.
explain_pair(0, 4, solvers_list=["1-1"])


In [ ]:
# Example 3: Extreme comparison (Rank 1 vs Rank 682 - The last one)
# This should be very easy to explain with simple (1-1) tradeoffs.
explain_pair(0, 681, solvers_list=["1-1"])
